라이브러리 불러오기

In [ ]:
# 라이브러리 설치
!pip install rouge-score transformers pandas

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=7db80baf4561b9e355c706b67db4651b9b5c376ce6e5b5ec6122de12da689926
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from google.colab import drive
import pandas as pd
import requests
import json
import time
import re
from rouge_score import rouge_scorer
from transformers import pipeline

In [ ]:
# 구글 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 엑셀 파일 읽기
file_path = '/content/drive/MyDrive/Colab Notebooks/미래에셋/CNBC_articles_Worldmarket.xlsx'  # 엑셀 파일 경로
df = pd.read_excel(file_path)

In [ ]:
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request, max_retries=3, delay=2):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'application/json'
        }

        for attempt in range(max_retries):
            try:
                response = requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                                         headers=headers, json=completion_request)
                response.raise_for_status()  # HTTP 에러가 발생했는지 확인
                return response.json()  # JSON 디코딩 시도
            except requests.exceptions.HTTPError as http_err:
                print(f'HTTP error occurred: {http_err}')
                if response.status_code == 429:  # Too Many Requests
                    print("Rate limit exceeded. Waiting before retrying...")
                    time.sleep(delay * (attempt + 1))  # 지연 시간 증가
                elif response.status_code == 500:  # Internal Server Error
                    print("Internal Server Error. Retrying...")
                    time.sleep(delay * (attempt + 1))  # 지연 시간 증가
                else:
                    break
            except requests.exceptions.RequestException as req_err:
                print(f'Request error occurred: {req_err}')
                break
            except json.JSONDecodeError as json_err:
                print(f'JSON decode error: {json_err}')
                print(f'Response content: {response.content}')
                break

        return None

In [ ]:
def summarize_text(text, executor):
    preset_text = [{
        "role": "system",
        "content": "- 당신은 텍스트 요약 어시스턴트입니다.\n- 주어진 텍스트를 분석하고 핵심 내용을 추출하여 간결하고 명확한 3줄 요약을 제공합니다.\n- 다음 지침을 엄격히 준수하여 업무를 수행하십시오.\n- 주어진 텍스트의 전체적인 맥락과 주제를 파악합니다.\n- 불필요한 세부사항은 제외하고 간결하고 명확한 문장으로 요약을 작성하세요.\n- 번호를 사용해 정보를 구조화하고, 모든 응답은 한국어로만 제공하세요.\n- 원문의 핵심을 정확히 반영하는 효과적인 요약을 만들어주세요.\n- 위의 지침을 철저히 따라 고품질의 텍스트 요약을 3줄로 제공합니다.\n\n###형식\n1. 내용 요약1\n2. 내용 요약2\n3. 내용 요약3\n###끝"
    }]

    completion_request = {
        'messages': preset_text + [{"role": "user", "content": text}],
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.1,
        'repeatPenalty': 1.2,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }

    response = executor.execute(completion_request)

    if response is not None:
        print("API Response:", response)  # 디버깅을 위해 응답 출력
        result = response.get('result', {})
        message = result.get('message', {})
        summary = message.get('content', '')
        summary = summary.replace('\n', ' ')  # 줄 바꿈 문자를 공백으로 대체
        return summary
    else:
        return '요약 실패'

In [ ]:
# 네이버 클라우드 API 설정
completion_executor = CompletionExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key='private_api_key',
    api_key_primary_val='private_api_key',
    request_id='private_api_key'
)

In [ ]:
# 첫 번째 행의 텍스트 요약
first_text = df.iloc[0, 2]  # 첫 번째 행의 3열 데이터 추출
print("Original Text:", first_text)  # 원문 텍스트 출력
summary = summarize_text(first_text, completion_executor)

Original Text: The dollar was on the front foot on Wednesday having rebounded from a three-week low after Federal Reserve Chair Jerome Powell struck a cautious tone on how soon interest rate cuts would come.

The dollar was firm against the euro on Monday as investors awaited further clues to help them figure out the path of U.S. interest rates in the wake of cautious comments from Federal Reserve officials, even as inflation showed signs of cooling.

Atlanta Fed President Raphael Bostic said on Monday it will take a while for the U.S. central bank to be confident that inflation is on track back to the central bank's 2% goal.

"The issue right now is when are we going to be certain that inflation is clearly on a path back to 2%. I think it's going to take a while before we know that for sure," Bostic said in an interview with Bloomberg Television.

Besides Bostic, several other Fed speakers are due to speak on Monday and later in the week. Markets will also focus on minutes of the Fed'

In [ ]:
# 요약 결과 출력
print("Summary:", summary)

Summary: 1. 달러는 제롬 파월 연방준비제도(Fed) 의장이 금리 인하 시기에 대해 신중한 태도를 보인 후 3주 최저치에서 반등하며 수요일 강세를 보였다. 2. 월요일 투자자들은 인플레이션이 둔화 조짐을 보이고 있음에도 불구하고 연방준비제도(Fed) 관계자들의 신중한 발언에 이어 미국 금리의 향방을 파악하기 위해 추가 단서를 기다리는 가운데 달러는 유로화에 대해 강세를 보였다. 3. 라파엘 보스틱 애틀랜타 연방준비은행 총재는 월요일 미국 중앙은행이 인플레이션이 중앙은행의 2% 목표치로 복귀하고 있다고 확신하는 데는 시간이 걸릴 것이라고 말했다.


3열의 text에 대해 적용

In [ ]:
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request, max_retries=3, delay=2):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'application/json'
        }

        for attempt in range(max_retries):
            try:
                response = requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                                         headers=headers, json=completion_request)
                response.raise_for_status()  # HTTP 에러가 발생했는지 확인
                return response.json()  # JSON 디코딩 시도
            except requests.exceptions.HTTPError as http_err:
                print(f'HTTP error occurred: {http_err}')
                if response.status_code == 429:  # Too Many Requests
                    print("Rate limit exceeded. Waiting before retrying...")
                    time.sleep(delay * (attempt + 1))  # 지연 시간 증가
                elif response.status_code == 500:  # Internal Server Error
                    print("Internal Server Error. Retrying...")
                    time.sleep(delay * (attempt + 1))  # 지연 시간 증가
                else:
                    break
            except requests.exceptions.RequestException as req_err:
                print(f'Request error occurred: {req_err}')
                break
            except json.JSONDecodeError as json_err:
                print(f'JSON decode error: {json_err}')
                print(f'Response content: {response.content}')
                break

        return None

In [ ]:
def summarize_text(text, executor):
    preset_text = [{
        "role": "system",
        "content": "- 당신은 텍스트 요약 어시스턴트입니다.\n- 주어진 텍스트를 분석하고 핵심 내용을 추출하여 간결하고 명확한 3줄 요약을 제공합니다.\n- 다음 지침을 엄격히 준수하여 업무를 수행하십시오.\n- 주어진 텍스트의 전체적인 맥락과 주제를 파악합니다.\n- 불필요한 세부사항은 제외하고 간결하고 명확한 문장으로 요약을 작성하세요.\n- 번호를 사용해 정보를 구조화하고, 모든 응답은 한국어로만 제공하세요.\n- 원문의 핵심을 정확히 반영하는 효과적인 요약을 만들어주세요.\n- 위의 지침을 철저히 따라 고품질의 텍스트 요약을 3줄로 제공합니다.\n\n###형식\n1. 내용 요약1\n2. 내용 요약2\n3. 내용 요약3\n###끝"
    }]

    completion_request = {
        'messages': preset_text + [{"role": "user", "content": text}],
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.1,
        'repeatPenalty': 1.2,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }

    response = executor.execute(completion_request)

    if response is not None:
        print("API Response:", response)  # 디버깅을 위해 응답 출력
        result = response.get('result', {})
        message = result.get('message', {})
        summary = message.get('content', '')
        summary = summary.replace('\n', ' ')  # 줄 바꿈 문자를 공백으로 대체
        return summary
    else:
        return '요약 실패'

In [ ]:
# 네이버 클라우드 API 설정
completion_executor = CompletionExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key='private_api_key',
    api_key_primary_val='private_api_key',
    request_id='private_api_key'
)

In [ ]:
# 전체 행의 텍스트 요약
texts_to_summarize = df.iloc[:, 2].tolist()  # 전체 행의 3열 데이터 추출
summaries = [summarize_text(text, completion_executor) for text in texts_to_summarize]

API Response: {'status': {'code': '20000', 'message': 'OK'}, 'result': {'message': {'role': 'assistant', 'content': '1. 달러는 제롬 파월 연방준비제도(Fed) 의장이 금리 인하 시기에 대해 신중한 태도를 보인 후 3주 최저치에서 반등하며 수요일 강세를 보였다.\n2. 월요일 투자자들은 인플레이션이 둔화 조짐을 보이고 있음에도 불구하고 연방준비제도(Fed) 관계자들의 신중한 발언에 이어 미국 금리의 향방을 파악하기 위해 추가 단서를 기다리는 가운데 달러는 유로화에 대해 강세를 보였다.\n3. 라파엘 보스틴 애틀랜타 연방준비은행 총재는 월요일 미국 중앙은행이 인플레이션이 중앙은행의 2% 목표치로 복귀하고 있다고 확신하는 데는 시간이 걸릴 것이라고 말했다.'}, 'inputLength': 939, 'outputLength': 136, 'stopReason': 'stop_before', 'seed': 442560105, 'aiFilter': [{'groupName': 'curse', 'name': 'insult', 'score': '2', 'result': 'OK'}, {'groupName': 'curse', 'name': 'discrimination', 'score': '2', 'result': 'OK'}, {'groupName': 'unsafeContents', 'name': 'sexualHarassment', 'score': '2', 'result': 'OK'}]}}
API Response: {'status': {'code': '20000', 'message': 'OK'}, 'result': {'message': {'role': 'assistant', 'content': '1. 이번 주 여러 청정 에너지 주식이 상승했으며, 일부 투자자들은 상승 원인에 대해 의문을 제기하고 있다.\n2. 레이몬드 제임스의 주식 분석가인 파벨 몰차노프는 화요일의 랠리가 미국이 중국에 관세

In [ ]:
# 요약 결과를 새로운 열에 추가
df['Summary'] = summaries

In [ ]:
df['Summary'].head()

0    1. 달러는 제롬 파월 연방준비제도(Fed) 의장이 금리 인하 시기에 대해 신중한 ...
1    1. 이번 주 여러 청정 에너지 주식이 상승했으며, 일부 투자자들은 상승 원인에 대...
2    목요일 유럽 증시는 예상보다 낮은 미국 인플레이션 데이터에 따른 글로벌 시장의 안도...
3    1. 금 가격은 목요일 달러가 반등하면서 소폭 하락했지만, 미국 인플레이션 둔화 신...
4    1. 미국의 인플레이션 완화와 금리 인하 가능성으로 인해 유가가 상승했다. 2. 미...
Name: Summary, dtype: object

3번째의 경우 넘버링이 되어 있지 않음.

In [ ]:
df['Summary'].tail()

195    1. 아시아 태평양 시장은 제롬 파월 연방준비제도(Fed) 의장의 비둘기파적 발언으...
196    1. 미국 연방준비제도 제롬 파월 의장의 발언으로 9월 금리 인하 가능성이 높아져 ...
197    1. 화요일 유럽 증시는 지역 및 전 세계 경제 및 정치 전망을 투자자들이 평가함에...
198    1. 화요일 국제유가는 중국의 경제 둔화로 수요가 감소할 것이라는 우려로 하락했지만...
199    1. 달러는 월요일 제롬 파월 연방준비제도(Fed) 의장의 발언 이후 소폭 하락한 ...
Name: Summary, dtype: object

In [ ]:
# 넘버링이 되어 있지 않은 경우만 추출
filtered_df = df[~df['Summary'].str.startswith('1.')]
filtered_df

,date,title,text,Summary
2,2024-05-16,Europe stocks close lower as earnings dampen r...,European stocks closed lower Thursday as earni...,목요일 유럽 증시는 예상보다 낮은 미국 인플레이션 데이터에 따른 글로벌 시장의 안도...
8,2024-05-17,"China stimulus, U.S. rate cut bets lift gold, ...",An employee handles one kilogram gold bullions...,"2023년 12월 22일 금요일, 태국 방콕에 있는 YLG Bullion Inter..."
33,2024-05-27,European stocks close higher; UK and U.S. mark...,European markets closed slightly higher on Mon...,월요일 유럽 시장은 조용한 시장에서 거래자들이 방향을 모색하면서 소폭 상승 마감했습...
73,2024-06-07,US dollar bounces as strong jobs report likely...,The dollar hovered close to an eight-week low ...,금요일 미국 달러는 연방준비제도(Fed·연준)의 금리 인하 시기에 대한 단서를 제공...
93,2024-06-13,"Dollar gains on hawkish Fed, even as inflation...",Asian currencies were firm on Thursday against...,목요일 아시아 통화는 미국 인플레이션이 예상보다 약해지면서 달러 대비 강세를 보였으...
163,2024-07-04,Oil prices slip after data points to cooling U...,Oil prices fell in early trade on Thursday aft...,목요일 미국 고용 및 기업 활동 데이터가 예상보다 부진하게 나오면서 세계 최대 원유...


In [ ]:
filtered_df['Summary']

2      목요일 유럽 증시는 예상보다 낮은 미국 인플레이션 데이터에 따른 글로벌 시장의 안도...
8      2023년 12월 22일 금요일, 태국 방콕에 있는 YLG Bullion Inter...
33     월요일 유럽 시장은 조용한 시장에서 거래자들이 방향을 모색하면서 소폭 상승 마감했습...
73     금요일 미국 달러는 연방준비제도(Fed·연준)의 금리 인하 시기에 대한 단서를 제공...
93     목요일 아시아 통화는 미국 인플레이션이 예상보다 약해지면서 달러 대비 강세를 보였으...
163    목요일 미국 고용 및 기업 활동 데이터가 예상보다 부진하게 나오면서 세계 최대 원유...
Name: Summary, dtype: object

In [ ]:
# 각 행별로 마침표 개수 세기
filtered_df['Period_Count'] = filtered_df['Summary'].str.count('\.')

<ipython-input-13-58f12aba8fa3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Period_Count'] = filtered_df['Summary'].str.count('\.')


In [ ]:
filtered_df['Period_Count']

2      14
8      28
33     16
73     24
93     20
163    13
Name: Period_Count, dtype: int64

=> 3문장도 아닌 것을 확인할 수 있음.

Rouge Recall 로 평가하기
- 넘버링(1. 2. 3.)이 되어있지 않은 문장은 0점 처리
- 넘버링(1. 2. 3.)이 되어있는 문장은 1점 처리

In [ ]:
# 조건 확인 함수 정의
def check_summary(summary):
    pattern = r'1\.\s.*2\.\s.*3\.\s.*'
    if re.search(pattern, summary):
        return 1
    else:
        return 0

In [ ]:
# 조건 확인
df["Check"] = df["Summary"].apply(check_summary)

In [ ]:
df['Check']

0      1
1      1
2      0
3      1
4      1
      ..
195    1
196    1
197    1
198    1
199    1
Name: Check, Length: 200, dtype: int64

In [ ]:
df_fail = df[df['Check'] == 0]
df_fail

,date,title,text,Summary,Check
2,2024-05-16,Europe stocks close lower as earnings dampen r...,European stocks closed lower Thursday as earni...,목요일 유럽 증시는 예상보다 낮은 미국 인플레이션 데이터에 따른 글로벌 시장의 안도...,0
8,2024-05-17,"China stimulus, U.S. rate cut bets lift gold, ...",An employee handles one kilogram gold bullions...,"2023년 12월 22일 금요일, 태국 방콕에 있는 YLG Bullion Inter...",0
33,2024-05-27,European stocks close higher; UK and U.S. mark...,European markets closed slightly higher on Mon...,월요일 유럽 시장은 조용한 시장에서 거래자들이 방향을 모색하면서 소폭 상승 마감했습...,0
73,2024-06-07,US dollar bounces as strong jobs report likely...,The dollar hovered close to an eight-week low ...,금요일 미국 달러는 연방준비제도(Fed·연준)의 금리 인하 시기에 대한 단서를 제공...,0
93,2024-06-13,"Dollar gains on hawkish Fed, even as inflation...",Asian currencies were firm on Thursday against...,목요일 아시아 통화는 미국 인플레이션이 예상보다 약해지면서 달러 대비 강세를 보였으...,0
163,2024-07-04,Oil prices slip after data points to cooling U...,Oil prices fell in early trade on Thursday aft...,목요일 미국 고용 및 기업 활동 데이터가 예상보다 부진하게 나오면서 세계 최대 원유...,0


=> filtered_df와 결과 일치

- 이를 통해 지정한 템플릿대로 출력하지 않은 결과가 전체 데이터셋 200개 중 6개인 것으로 나타났다.
- 따라서 올바르게 출력한 결과는 전체 데이터셋 200개 중 194개인 것으로 나타났다.
- 이를 통해 HyperClova X의 해외금융뉴스요약기 프롬프트의 정확도가 97%임을 알 수 있다.